# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [4]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release)]
        trad_releases = [release for release in project.main_releases if is_trad_release(release)]

        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        for srelease in project.main_releases:
            release = srelease.release
            cycle_days = delta2days(srelease.cycle)
            delay = delta2days(srelease.delay)
            contributors_top = release.contributors.top(percent=50)
            contributors_newcomers = release.contributors.newcomers
            
            if srelease.commits:
                development = delta2days(srelease.time - srelease.commits.first(lambda c: c.author_time).author_time)
                if srelease.base_release:
                    srelease_delta = srelease.release.head.diff(srelease.base_release.release.head)
                else:
                    first_commit = srelease.commits.first(lambda c: c.author_time)
                    srelease_delta = srelease.release.head.diff(srelease.commits.first(lambda c: c.author_time))
                srelease_churn = srelease_delta.churn
            else:
                srelease_churn = 0
                development = 0 
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle': srelease.cycle,
                'cycle_days': cycle_days,
                'cycle_rdays': math.ceil(cycle_days),
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'contributors': len(release.contributors),
                'contributors_top': len(contributors_top),
                'contributors_newcomers': len(contributors_newcomers),
                
                'commits': len(release.commits),
                'commits_top': len(release.contributors.commits(contributors_top)),
                'commits_newcomers': len(release.contributors.commits(contributors_newcomers)),
                
                'churn_release': srelease_churn,
                'delay': delay,
                'delay_rdays': math.ceil(delay),
                'development': development,
                'development_rdays': math.ceil(development),
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [5]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [ ]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1665)

In [ ]:
projects = pd.DataFrame(project_results)
projects.sample(10)

In [ ]:
releases = pd.DataFrame(release_results)
releases.sample(10)

In [ ]:
len(projects)

In [ ]:
len(releases)

In [ ]:
len(releases.query("rapid_release == True"))

In [ ]:
len(releases.query("trad_release == True"))

In [ ]:
len(releases.query("rapid_release == False and trad_release == False"))

In [ ]:
releases['commits'].sum()

In [ ]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [ ]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)